In [102]:
# If don't have, install only once
# !pip install kagglehub

import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("snehangsude/audible-dataset")

df = pd.read_csv(os.path.join(path, 'audible_uncleaned.csv'))

# Check first views:
# df.head()
# df.describe()
# df.info()

In [103]:
# Clean text data in Author and Narrator columns
# Remove Writtenby: from the author column
df['author'] = df['author'].str.replace('Writtenby:', '')
# Remove Narratedby: from the narrator column
df['narrator'] = df['narrator'].str.replace('Narratedby:', '')
# df.head(2)

In [104]:
# Clean stars table
# Create separate column for ratings
df['ratings'] = df['stars'].str.extract(r'stars(\d+)\s')
df['ratings'] = df['ratings'].fillna(0).astype('int')
# Create separate column for stars
df['stars out of five'] = df['stars'].str.extract(r'(\d+)\sout')
df['stars out of five'] = df['stars out of five'].fillna(0).astype('float')
df.drop('stars', axis=1, inplace=True)
# df.head(2)
# df['stars out of five'].unique()
# df['ratings'].sample(n=30)

In [105]:
# Replace the comma with ''
df['price'] = df['price'].str.replace(',', '')
# Replace 'Free' with 0
df['price'] = df['price'].str.replace('Free', '0')
# Turn price to float
df['price'] = df['price'].astype(float)
# df['price'].sample(n=30)

In [106]:
# Convert releasedate to datetime
df['releasedate'] = pd.to_datetime(df['releasedate'])
# Inspect the dataframe 
# df.info()
# Search the entries in the time column for different spellings of min. Let' try min, mins, minutes
# df['time'].str.contains(r'\b(min|mins|minutes)\b', case=False, na=False)

# Replace hrs, mins, and 'Less than 1 minute'
df['time'] = df['time'].str.replace(r'\b(mins?|minutes?)\b', 'min', case=False, regex=True)
df['time'] = df['time'].str.replace(r'\b(hr?|hrs?)\b', 'hr', case=False, regex=True)
df['time'] = df['time'].str.replace('Less than 1 minute', '1 min')
# set(type(x) for x in df['time'])
# df['time'].isna().any()

# Extract the number of hours, turn to integer
df['hours'] = df['time'].str.extract(r'(\d+)\s*hr')
df['hours'] = df['hours'].fillna("0").astype('str')
# Extract the number of minutes, turn to integer
df['mins'] = df['time'].str.extract(r'and\s(\d+)\s*min')
df['mins'] = df['mins'].fillna("0").astype('int')
df['mins'] = df['mins'].apply(lambda x: f"{x:02}")
# Combine hours and minutes into the duration column
df['duration'] = df['hours'].astype('str') + ':' + df['mins'].astype('str')

df.drop('mins', axis=1, inplace=True)
df.drop('hours', axis=1, inplace=True)
# Check the results
df.head()

/tmp/ipykernel_1382/1318010996.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['releasedate'] = pd.to_datetime(df['releasedate'])


,name,author,narrator,time,releasedate,language,price,ratings,stars out of five,duration
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hr and 20 min,2008-04-08,English,468.0,34,5.0,2:20
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hr and 8 min,2018-01-05,English,820.0,41,5.0,13:08
2,The Deep End,JeffKinney,DanRussell,2 hr and 3 min,2020-06-11,English,410.0,38,5.0,2:03
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hr and 16 min,2021-05-10,English,615.0,12,5.0,11:16
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hr,2010-01-13,English,820.0,181,5.0,10:00


In [ ]:
# Transform prices to USD (multiply times 0.012)
df['price'] = df['price'] * 0.012
# Check the results
# df['price'].sample(n=30)

49812    21.096
52042     4.812
24143     6.180
16619     6.696
61376     6.468
51653    10.032
84542     8.436
81093     3.216
6957      1.788
75889     7.032
18476     8.436
68997    13.776
7887      2.400
26583    14.064
11055     8.196
80205    11.256
79773     8.016
27121     5.436
10432     8.436
40902    12.036
34745     1.992
69347     3.660
63998     6.192
73942     3.876
86281     9.108
35360     7.584
54578    14.064
66982     5.616
23901     4.008
80723     6.024
Name: price, dtype: float64

In [ ]:
# Update capitalization in the language column
df['language'] = df['language'].str.capitalize()
# Check the results
# df['language'].sample(n=40)

26920     English
71290     English
65968     English
25496     English
21749     English
16019    Japanese
66470     English
17892     English
68828    Japanese
12055     Russian
5212      English
38729     English
44255     English
55852     Russian
10277     English
75755     English
86202       Hindi
66723     English
82339     English
52485      German
2369      English
82146     English
44014     English
37386     English
66848     English
47635     Spanish
36708     English
10375     English
28880     English
87329      German
46525     English
53975     Spanish
14561      French
13591     English
612       Spanish
37152     English
4351       German
24935     English
1006      English
72208     English
Name: language, dtype: object

In [ ]:
# Checking the duplicates
duplicates = df.duplicated().sum()
print(duplicates)


5
